In [ ]:
# =========================================================================
# JUPYTER CELL 1: ИНИЦИАЛИЗАЦИЯ, ФУНКЦИИ И ПАРАМЕТРЫ
# Содержит всю логику для обучения и дообучения
# =========================================================================

# =========================================================================
# БЛОКНОТ ДЛЯ ДООБУЧЕНИЯ (FINE-TUNING) АГЕНТОВ
# =========================================================================

import sys
import os
from pathlib import Path

# Правильная настройка путей
current_dir = Path.cwd()
if current_dir.name == "notebooks":
    project_root = current_dir.parent
    sys.path.insert(0, str(project_root))
else:
    project_root = Path(os.getcwd())
    sys.path.insert(0, str(project_root))

import pandas as pd
from training import TrainingManager, TrainingConfig

# --- 1. ГЛОБАЛЬНЫЕ ПАРАМЕТРЫ ВЕРСИИ И ЭПИЗОДОВ ---
TRAIN_VERSION = "v3"  # Должно совпадать с версией в test_train.ipynb
N_EPISODES_INITIAL = 5000   
N_EPISODES_FINETUNE = 2000  # Количество эпизодов на втором этапе
N_EPISODES_START_FINETUNE = N_EPISODES_INITIAL 
N_EPISODES_END_FINETUNE = N_EPISODES_INITIAL + N_EPISODES_FINETUNE


# --- 2. ЗАГРУЗКА И РАЗДЕЛЕНИЕ ДАННЫХ ---
data_path_2021 = project_root / "data" / "data_1h_2021.csv"
data_path_2022 = project_root / "data" / "data_1h_2022.csv"
data_path_2023 = project_root / "data" / "data_1h_2023.csv"

if not data_path_2021.exists():
    data_path_2021 = project_root.parent / "data" / "data_1h_2021.csv"
    data_path_2022 = project_root.parent / "data" / "data_1h_2022.csv"
    data_path_2023 = project_root.parent / "data" / "data_1h_2023.csv"

df1 = pd.read_csv(data_path_2021, index_col=0, parse_dates=True, date_format="iso8601")
df2 = pd.read_csv(data_path_2022, index_col=0, parse_dates=True, date_format="iso8601")
df3 = pd.read_csv(data_path_2023, index_col=0, parse_dates=True, date_format="iso8601") if data_path_2023.exists() else None


# Данные для первичного обучения (2021 + 1/2 2022)
df_train = pd.concat([df1, df2.iloc[:len(df2)//2]])

# Данные для дообучения (1/2 2022 + 2023)
if df3 is not None:
    df_finetune = pd.concat([df2.iloc[len(df2)//2:], df3])
else:
    df_finetune = df2.iloc[len(df2)//2:]

print(f"Данные для дообучения: {len(df_finetune)} строк")
print(f"Период: {df_finetune.index[0]} - {df_finetune.index[-1]}")


# --- 3. ПАРАМЕТРЫ СРЕДЫ (ENV PARAMS) ---
env_params = {
    "initial_balance": 1000.0,
    "window_size": 10,
    "commission": 0.0001,
    "slippage": 0.0001,
    "max_holding_time": 48,
    "holding_threshold": 12, 
    "max_drawdown_threshold": 0.05,
    "lambda_drawdown": 0.5,
    "lambda_hold": 0.1,
    "reward_scaling": 1.0, 
    "max_steps": 1000
}

# --- 4. ПАРАМЕТРЫ ОБУЧЕНИЯ (TRAIN PARAMS) ---

# 4.1. Параметры Первичного Обучения
base_train_params = {
    "n_episodes": N_EPISODES_INITIAL,
    "n_episodes_start": 0,
    "max_steps": 1000,
    "eval_frequency": 100,
    "save_frequency": 500,
    "patience": 200,
    "initial_balance": 1000.0,
    **{k: env_params[k] for k in env_params if k != "max_steps"} 
}

# 4.2. Параметры Дообучения
finetune_train_params = {
    "n_episodes": N_EPISODES_END_FINETUNE, # 4500
    "n_episodes_start": N_EPISODES_START_FINETUNE, # 3000
    "max_steps": 1000,
    "eval_frequency": 50, # Увеличиваем частоту оценки
    "save_frequency": 500,
    "patience": 200,
    "initial_balance": 1000.0,
    **{k: env_params[k] for k in env_params if k != "max_steps"}
}

# 4.3. Конфигурации Агентов (Гиперпараметры)
agents_config = {
    "QLearning": {
        "learning_rate": 0.05, "discount_factor": 0.99,
        "epsilon_start": 1.0, "epsilon_end": 0.01, "epsilon_decay": 0.998,
    },
    "SARSA": {
        "learning_rate": 0.05, "discount_factor": 0.99,
        "epsilon_start": 1.0, "epsilon_end": 0.01, "epsilon_decay": 0.998,
    },
    "SARSA_Lambda": {
        "learning_rate": 0.02, "discount_factor": 0.99,
        "epsilon_start": 1.0, "epsilon_end": 0.01, "epsilon_decay": 0.998,
        "trace_decay": 0.9,
    },
    "Monte_Carlo": {
        "learning_rate": 0.02, "discount_factor": 0.99,
        "epsilon_start": 1.0, "epsilon_end": 0.05,  "epsilon_decay": 0.999, 
    }
}


# --- 5. ФУНКЦИИ ---

def _get_agent_instance(agent_type: str, hyperparams: dict):
    """Создает экземпляр агента и присваивает гиперпараметры."""
    if agent_type == "SARSA":
        from agents.classical.sarsa_agent import SarsaAgent
        agent = SarsaAgent()
    elif agent_type == "SARSA_Lambda":
        from agents.classical.sarsa_lambda_agent import SarsaLambdaAgent
        agent = SarsaLambdaAgent()
    elif agent_type == "QLearning":
        from agents.classical.qlearning_agent import QLearningAgent
        agent = QLearningAgent()
    elif agent_type == "Monte_Carlo":
        from agents.classical.monte_carlo_agent import MonteCarloAgent
        agent = MonteCarloAgent()
    else:
        raise ValueError(f"Unknown agent type: {agent_type}")
        
    for key, value in hyperparams.items():
        if hasattr(agent, key):
             setattr(agent, key, value)
    return agent


def run_training(
    agent_type: str,
    df,
    project_root: str,
    env_params: dict,
    train_params: dict,
):
    """Запускает первичное обучение для одного агента."""
    hyperparams = agents_config.get(agent_type, {})
    agent = _get_agent_instance(agent_type, hyperparams)

    config = TrainingConfig(
        agent_name=f"{agent_type}_{TRAIN_VERSION}",
        agent_type=agent_type,
        **train_params,
    )

    manager = TrainingManager(
        base_log_dir=os.path.join(project_root, "training_data/logs"),
        base_checkpoint_dir=os.path.join(project_root, "training_data/checkpoints"),
    )

    experiment_name = f"exp_{agent_type.lower()}_{TRAIN_VERSION}"
    print(f"\n=======================================================")
    print(f" 🚀 НАЧАЛО ПЕРВИЧНОГО ОБУЧЕНИЯ: {agent_type} (Ep: 0 -> {train_params['n_episodes']})")
    print(f"=======================================================")
    return manager.train_agent(agent, df, config, experiment_name)


def run_finetuning(
    agent_type: str,
    df,
    project_root: str,
    env_params: dict,
    train_params: dict, # finetune_train_params
):
    """Запускает дообучение (Fine-tuning) с загрузкой чекпоинта."""

    hyperparams = agents_config.get(agent_type, {})
    
    # 1. Пути к сохраненному агенту (из exp_{agent}_v2)
    experiment_name_base = f"exp_{agent_type.lower()}_{TRAIN_VERSION}"
    experiment_name_finetune = f"{experiment_name_base}_finetune"
    
    checkpoint_path = os.path.join(
        project_root, "training_data", "checkpoints", experiment_name_base, "final_agent.pkl",
    )

    if not Path(checkpoint_path).exists():
        print(f"!!! ОШИБКА: Файл чекпоинта не найден: {checkpoint_path}")
        raise FileNotFoundError("Сначала запустите и завершите ПЕРВИЧНОЕ ОБУЧЕНИЕ.")

    # 2. Создание агента
    agent = _get_agent_instance(agent_type, hyperparams) 

    # 3. Конфигурация дообучения (указывает на продолжение нумерации эпизодов)
    continue_config = TrainingConfig(
        agent_name=f"{agent_type}_{TRAIN_VERSION}_finetune",
        agent_type=agent_type,
        **train_params,
    )

    # 4. Менеджер и запуск
    manager = TrainingManager(
        base_log_dir=os.path.join(project_root, "training_data/logs"),
        base_checkpoint_dir=os.path.join(project_root, "training_data/checkpoints"),
    )
    
    print(f"\n=======================================================")
    print(f" ♻️ НАЧАЛО ДООБУЧЕНИЯ: {agent_type} (Ep: {train_params['n_episodes_start']} -> {train_params['n_episodes']})")
    print(f"=======================================================")
    
    return manager.continue_training(
        agent=agent,
        df=df,
        config=continue_config,
        checkpoint_path=checkpoint_path,
        experiment_name=experiment_name_finetune,
        verbose=True,
    )

In [ ]:
# =========================================================================
# JUPYTER CELL 2: ЗАПУСК ДООБУЧЕНИЯ (FINE-TUNING)
# Запускать после завершения всех тренировок из Части 1
# =========================================================================

# Убедитесь, что Ячейка 1 выполнена!

for agent_name in agents_config.keys():
    try:
        run_finetuning(
            agent_type=agent_name,
            df=df_finetune,
            project_root=project_root,
            env_params=env_params,
            train_params=finetune_train_params,
        )
    except FileNotFoundError:
        print(f"--- Пропуск дообучения {agent_name}: Не найден базовый чекпоинт. ---")
        continue
# Чтобы запустить только один агент, используйте:
# run_finetuning(
#     agent_type="QLearning",
#     df=df_finetune,
#     project_root=project_root,
#     env_params=env_params,
#     train_params=finetune_train_params,
# )
